In [14]:
from multiprocessing import Pool
from IPython.display import clear_output
import os
import argparse


import jieba
import pandas as pd
import numpy as np


from procDataSet import TrainingQuery


top_num = 300

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設', '月退']]

# add NTUSD
positive_words = np.squeeze(pd.read_csv('data/NTUSD/ntusd-positive_zhtw.txt', header=None).values).tolist()
negative_words = np.squeeze(pd.read_csv('data/NTUSD/ntusd-negative_zhtw.txt', header=None).values).tolist()
for w in positive_words + negative_words:
    jieba.add_word(w)
    
def histogram_equalization(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, density=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 100 * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)
    return image_equalized

In [2]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [3]:
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [4]:
# add back some words
stop_words = list(set(stop_words) - set(negative_words) - set(positive_words))

In [5]:
def find_sentiment(cuts):
    p = n = 0
    for c in cuts:
        p += c in positive_words
        n += c in negative_words
    return p - n

for q in test_query:
    cuts = jieba.lcut(q)
    cuts = [c for c in cuts if c not in stop_words]
    print(cuts)
    print(find_sentiment(cuts))

['通姦', '刑法', '應該', '除罪', '化']
-1
['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉']
-3
['支持', '博弈', '特區', '台灣', '合法化']
0
['中華', '航空', '空服員', '罷工', '合理的']
0
['性交易', '應該', '合法化']
0
['ECFA', '早收', '清單', '達到', '預期', '成效']
1
['應該', '減免', '證所稅']
0
['贊成', '中油', '觀塘', '興建', '第三', '天然氣', '接收站']
1
['支持', '中國', '學生', '納入', '健保']
0
['支持', '臺灣', '中小學', '含', '高職', '專科', '服儀', '規定', '含', '髮', '襪', '鞋', '給予', '學生', '自主']
2
['不支持', '使用', '加密', '貨幣']
0
['不支持', '學雜費', '調漲']
0
['同意', '政府', '舉債', '發展', '前瞻建設', '計畫']
2
['支持', '電競', '列入', '體育競技']
0
['反對', '台鐵', '東移', '徵收', '案']
-2
['支持', '陳', '前', '總統', '保外就醫']
0
['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
0
['同意', '動物', '實驗']
1
['油價', '應該', '凍漲', '緩漲']
0
['反對', '旺旺', '中時', '併購', '中嘉']
-1


In [ ]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentence_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [6]:
sentence_arr = np.load('data/sentence_arr.npy')
# np.save('data/sentence_arr', sentence_arr)

## add NTUSD scoring

In [163]:
# 要跑超久
pool = multiprocessing.Pool(processes=NumberCPU)
sentiment_arr = pool.map(find_sentiment, sentence_arr)
pool.close()
pool.join()


In [7]:
# np.save('data/sentiment_arr', sentiment_arr)
sentiment_arr = np.load('data/sentiment_arr.npy')

## okapi model

In [8]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentence_arr)

In [9]:
train_query = sorted(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]

    # sort by relvanance
    y = np.squeeze(data[data.Relevance!=0].sort_values('Relevance', ascending=False).News_Index.values).tolist()
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]
    
    print(i, len(y))

十二年國教高中職「免學費補助」適用對象增加是不對的 93
反對二代健保規定 232
反對旺旺中時併購中嘉 109
反對無圍牆校園 35
另立專法保障同婚是正確的 135
同意動物實驗 20
國際賽事會場內應該可以持中華民國國旗 766
堅決反對政府舉債發展前瞻建設計畫 388
年金改革應取消或應調降軍公教月退之優存利率十八趴 200
應該提高酒駕罰責以有效遏制酒駕 441
拒絕公投通過門檻下修 52
支持正名「臺灣」參與國際運動賽事 110
支持陳前總統保外就醫 200
核四應該啟用 205
油價應該凍漲或緩漲 68
臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 248
贊同課綱微調 310
贊成文林苑都更案可依法拆除王家 65
贊成流浪動物零撲殺 270
遠雄大巨蛋工程應停工或拆除 240


In [26]:
total_scores = []
results = []
total_recall = []

for test_id , text_q in enumerate(train_query):

    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    scores = histogram_equalization(scores)
    
    top_query = np.argsort(scores)[::-1][:5]
    irrelevant_query = np.argsort(scores)[:5]
    
    print(test_id ,text_q, text)
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:200]
        top_words = [x[0] for x in top_words]
        text += top_words
        
    expansion_score = np.array(bm25Model.get_scores(text))
    expansion_score = histogram_equalization(expansion_score)
    scores += 5 * expansion_score
    sentiment_alpha = find_sentiment(text)
        
    if sentiment_alpha != 0:
#         sentiment_matching = sentiment_alpha * sentiment_arr  # MAP = 0.05
        sentiment_matching = sentiment_alpha * np.sign(sentiment_arr)  # MAP = 0.13
        scores += histogram_equalization(sentiment_matching)   # 多了 .5 scaling : MAP = 0.14
    
    # scoring
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    results.append(keys[0])
    
    count = 0
    ap = 0
    total = min([top_num, len(y_train[test_id])])
    for i in range(top_num):
        r = ans[i] in y_train[test_id]
        count += r
        p = count / (i+1)
        ap += p * r
    ap /= total
    rc = count / total
    print("Total: {}, AP: {:.2f}, recall: {:.2f}".format(total, ap, rc))
    total_scores += [ap]
    total_recall += [rc]

print('MAP: {:.2f}, avg recall: {:.2f}'.format(np.average(total_scores), np.average(total_recall)))

0 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中', '職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
Total: 93, AP: 0.17, recall: 0.59
1 反對二代健保規定 ['反對', '二代', '健保', '規定']
Total: 232, AP: 0.39, recall: 0.64
2 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
Total: 109, AP: 0.17, recall: 0.61
3 反對無圍牆校園 ['反對', '圍牆', '校園']
Total: 35, AP: 0.21, recall: 0.54
4 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確的']
Total: 135, AP: 0.13, recall: 0.44
5 同意動物實驗 ['同意', '動物', '實驗']
Total: 20, AP: 0.01, recall: 0.15
6 國際賽事會場內應該可以持中華民國國旗 ['國際', '賽事', '會場', '應該', '持', '中華民國', '國旗']
Total: 300, AP: 0.03, recall: 0.17
7 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
Total: 300, AP: 0.29, recall: 0.47
8 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
Total: 200, AP: 0.04, recall: 0.26
9 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒駕', '罰責以', '有效', '遏制', '酒駕']
Total: 300, AP: 0.44, recall: 0.64
10 拒絕公投通過門檻下修 ['拒絕', '公投', '通過', '門檻', '下修']
Total: 52, AP: 0.06, recall: 0.46
11 支持正名「臺灣

In [24]:
# 原本的
total_scores = []
results = []
total_recall = []

for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    scores = histogram_equalization(scores)
    
    top_query = np.argsort(scores)[::-1][:5]
    irrelevant_query = np.argsort(scores)[:5]
    
    print(test_id ,text_q, text)
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:200]
        top_words = [x[0] for x in top_words]
        text += top_words
        
    expansion_score = np.array(bm25Model.get_scores(text))
    expansion_score = histogram_equalization(expansion_score)
    scores += 5 * expansion_score
    
    # scoring
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    results.append(keys[0])
    
    count = 0
    ap = 0
    total = min([top_num, len(y_train[test_id])])
    for i in range(top_num):
        r = ans[i] in y_train[test_id]
        count += r
        p = count / (i+1)
        ap += p * r
    ap /= total
    rc = count / total
    print("Total: {}, AP: {:.2f}, recall: {:.2f}".format(total, ap, rc))
    total_scores += [ap]
    total_recall += [rc]
            
print('MAP: {:.2f}, avg recall: {:.2f}'.format(np.average(total_scores), np.average(total_recall)))

0 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中', '職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
top query num 5
Total: 93, AP: 0.17, recall: 0.61
1 反對二代健保規定 ['反對', '二代', '健保', '規定']
top query num 5
Total: 232, AP: 0.33, recall: 0.59
2 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 5
Total: 109, AP: 0.18, recall: 0.64
3 反對無圍牆校園 ['反對', '圍牆', '校園']
top query num 5
Total: 35, AP: 0.37, recall: 0.86
4 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確的']
top query num 5
Total: 135, AP: 0.12, recall: 0.45
5 同意動物實驗 ['同意', '動物', '實驗']
top query num 5
Total: 20, AP: 0.20, recall: 1.00
6 國際賽事會場內應該可以持中華民國國旗 ['國際', '賽事', '會場', '應該', '持', '中華民國', '國旗']
top query num 5
Total: 300, AP: 0.09, recall: 0.30
7 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
top query num 5
Total: 300, AP: 0.23, recall: 0.44
8 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
top query num 5
Total: 200, AP: 0.05, recall: 0.27
9 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒駕', '罰責以'

In [23]:
# 原本的
total_scores = []
results = []
total_recall = []

for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    scores = histogram_equalization(scores)
    
    top_query = np.argsort(scores)[::-1][:50]
    
    print(test_id ,text_q, text)
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:200]
        top_words = [x[0] for x in top_words]
        text += top_words
        
    expansion_score = np.array(bm25Model.get_scores(text))
    expansion_score = histogram_equalization(expansion_score)
    scores += 5 * expansion_score
    
    # scoring
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    results.append(keys[0])
    
    count = 0
    ap = 0
    total = min([top_num, len(y_train[test_id])])
    for i in range(top_num):
        r = ans[i] in y_train[test_id]
        count += r
        p = count / (i+1)
        ap += p * r
    ap /= total
    rc = count / total
    print("Total: {}, AP: {:.2f}, recall: {:.2f}".format(total, ap, rc))
    total_scores += [ap]
    total_recall += [rc]
            
print('MAP: {:.2f}, avg recall: {:.2f}'.format(np.average(total_scores), np.average(total_recall)))

0 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中', '職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
top query num 50
Total: 93, AP: 0.15, recall: 0.63
1 反對二代健保規定 ['反對', '二代', '健保', '規定']
top query num 50
Total: 232, AP: 0.27, recall: 0.57
2 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 50
Total: 109, AP: 0.18, recall: 0.68
3 反對無圍牆校園 ['反對', '圍牆', '校園']
top query num 50
Total: 35, AP: 0.41, recall: 0.89
4 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確的']
top query num 50
Total: 135, AP: 0.11, recall: 0.44
5 同意動物實驗 ['同意', '動物', '實驗']
top query num 50
Total: 20, AP: 0.24, recall: 0.95
6 國際賽事會場內應該可以持中華民國國旗 ['國際', '賽事', '會場', '應該', '持', '中華民國', '國旗']
top query num 50
Total: 300, AP: 0.09, recall: 0.31
7 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
top query num 50
Total: 300, AP: 0.21, recall: 0.42
8 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
top query num 50
Total: 200, AP: 0.07, recall: 0.32
9 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒

## Test the model

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:80]
    
    sentiment_alpha = find_sentiment(text)
    
    print(test_id ,text_q, text, sentiment_alpha)
    print("top query num {}".format(len(top_query)))
    
    # query expension
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
    if sentiment_alpha != 0:
        sentiment = sentiment_alpha * np.sign(sentiment_arr)
        scores += sentiment / np.amax(np.abs(sentiment)) * np.amax(scores) * .5

    # add train data
    delta = np.max(scores) 
    if text_q in y_index.keys():
        for idx, rel in (y_index[text_q][0]):
            scores[idx] += delta * rel
            
    total_scores.append(scores)

0 通姦在刑法上應該除罪化 ['通姦', '刑法', '應該', '除罪', '化'] -1
top query num 80
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉'] -3
top query num 80
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '台灣', '合法化'] 0
top query num 80
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '合理的'] 0
top query num 80


In [24]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [25]:
fname = '0625-2.csv'
df.to_csv('output/' + fname, index=False)